In [6]:
# Function for variables
def variables_decl(month_par, year_par):
    #Packages
    from datetime import timedelta
    from datetime import datetime
    # Variable declaration
    a = timedelta(seconds=300)
    month = str(month_par)      # Cast to string in order to use it in image titles (does not accept int types)
    year = str(year_par)    # Cast to string in order to use it in image titles (does not accept int types)

    return (a, month,year)

In [7]:
# Function for database import
def db_import():
    #Packages
    import cx_Oracle
    import pandas as pd
    
    # Initializing database connection
    con = cx_Oracle.connect('STC_SETRAG/art@localhost/XE')
    cur = con.cursor()

    # First Query
    query=cur.execute('select * from historico_mensagens')
    df = pd.DataFrame(query)
    # Second Query
    query=cur.execute('select EV_NOM_MAC from ELEM_VIA where TE_ID_TP = 3')
    df_sb = pd.DataFrame(query)

    cur.close()
    con.close()
    
    return (df,df_sb)

In [8]:
# Function for formatting dataframe
def df_format(df, df_sb):
    # Adding names to the columns of the dataframes
    df.columns=['HM_ID_HM','HM_LOCO_MSG','HM_PRF_TRM','HM_OBC_MSG','HM_NUM_MSG','HM_TP_MSG','HM_DAT_OBC','HM_DAT_STC',
                'HM_TXT_MSG','HM_TXT_TW','HM_LAT_HM','HM_LON_HM','HM_ENV_RCB','HM_MSG_VOZ','HM_NUM_FUN','HM_ID_AUX','HM_SIT_ENV','MT_ID_MT']
    df_sb.columns=['NAME']
    # Dropping tables with no use
    df = df.drop(['HM_TP_MSG','HM_TXT_TW','MT_ID_MT','HM_ID_AUX','HM_OBC_MSG','HM_LAT_HM','HM_LON_HM'], axis=1)
    
    return(df, df_sb)

In [9]:
# Function for splitting dataframe in 'Sent' and 'Received' messages
def df_split(df):
    #Packages
    import pandas as pd
    
    df_sent = df.loc[df['HM_ENV_RCB']=='E']
    df_received = df.loc[df['HM_ENV_RCB']=='R']

    # Removing the original dataframe
    df = pd.DataFrame(None)
    
    return(df, df_sent, df_received)

In [10]:
# Functing for first filtering (part I)
def filter_I (df_sent,df_received):
    # Sent DataFrame
    # 1) Filter messages from january 2021
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']>='2021-01-01']
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']<='2021-01-31']
    # 2) Filter all messages that contain OBC time
    df_sent = df_sent.loc[df_sent['HM_DAT_OBC'].notnull()]

    # Received DataFrame
    # 1) Filter all messages that did not expired
    df_received = df_received.loc[df_received['HM_SIT_ENV']=='E']
    # 2) Filter messages from january 2021
    df_received = df_received.loc[df_received['HM_DAT_STC']>='2021-01-01']
    df_received = df_received.loc[df_received['HM_DAT_STC']<='2021-01-31']
    # 3) Filter all messages that contain OBC time
    df_received = df_received.loc[df_received['HM_DAT_OBC'].notnull()]
    # 4) Filter all messages that have a date later than needed (avoiding future date messages)
    df_received = df_received.loc[df_received['HM_DAT_OBC']<'2021-02-02']
    
    return(df_sent, df_received)